In [1]:
%reload_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
## Environment
# Change main system path to be able to run code from src folder
import sys
p = sys.path[0]
# Mac OS
if sys.path[0].endswith('/models'):
    main_path = p[:-len('/models')]
sys.path[0] = main_path

import os, gc
from termcolor import colored
import pandas as pd
from src import config

In [3]:
include_val_loss = False
include_sam = False

In [4]:

if include_val_loss:
    cvloss_files = [each for each in os.listdir(config.MODELS_DIR) if (each.endswith('_cvloss.csv') | 
                                                                    each.endswith('_Vloss.csv'))]
else:
    cvloss_files = [each for each in os.listdir(config.MODELS_DIR) if (each.endswith('_cvloss.csv'))]
    
print(len(cvloss_files))

4


In [5]:
model_summary = pd.DataFrame()
for i in cvloss_files:
    df = pd.read_csv(os.path.join(config.MODELS_DIR, i), index_col='target')
    model_summary = pd.concat([model_summary, df], axis=1)

target_labels_list = model_summary.index.tolist()

In [6]:
model_summary.style.highlight_min(axis=1,
                                  props='color:darkblue;\
                                         background-color:lightblue;')

,fts_mra_tempmz_XGB_opt_tr_sfm,fts_mra_tempmz_LR_reg_tr,fts_mra_tempmz_slope_XGB_opt_tr_sfm,fts_mra_tempmz_XGB_opt_tr
target,,,,
basalt,0.203793,0.276783,0.191178,0.196651
carbonate,0.108804,0.261255,0.109929,0.121020
chloride,0.209937,0.284298,0.207700,0.204338
iron_oxide,0.224210,0.382470,0.220018,0.226116
oxalate,0.025980,0.004059,0.025988,0.029273
oxychlorine,0.180170,0.299150,0.177872,0.194202
phyllosilicate,0.245376,0.407476,0.246543,0.259516
silicate,0.233179,0.351685,0.233784,0.233179
sulfate,0.190160,0.344719,0.184260,0.203174


In [7]:
model_sum = model_summary.T.copy()
model_sum['overall'] = model_sum.mean(axis=1)
if not include_sam:
    model_sum = model_sum[model_sum.index.isin([i for i in model_sum.index if not i.endswith('_sam')])]
model_sum.sort_values(by='overall')\
             .style.highlight_min(axis=0, 
                                  props='color:darkblue; background-color:lightblue;',
                                  subset=target_labels_list)\
                    .highlight_min(axis=0, 
                                   props='color:white; background-color:red;',
                                   subset=['overall'])

target,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide,overall
fts_mra_tempmz_slope_XGB_opt_tr_sfm,0.191178,0.109929,0.207700,0.220018,0.025988,0.177872,0.246543,0.233784,0.184260,0.067739,0.166501
fts_mra_tempmz_XGB_opt_tr_sfm,0.203793,0.108804,0.209937,0.224210,0.025980,0.180170,0.245376,0.233179,0.190160,0.067537,0.168915
fts_mra_tempmz_XGB_opt_tr,0.196651,0.121020,0.204338,0.226116,0.029273,0.194202,0.259516,0.233179,0.203174,0.078807,0.174628
fts_mra_tempmz_LR_reg_tr,0.276783,0.261255,0.284298,0.382470,0.004059,0.299150,0.407476,0.351685,0.344719,0.147136,0.275903


In [ ]:
dt = model_sum[~model_sum.index.isin([i for i in model_sum.index if i.endswith('_trvl')])].sort_index()
dt.style.highlight_min(axis=0, 
                        props='color:darkblue; background-color:lightblue;',
                        subset=target_labels_list)\
        .highlight_min(axis=0, 
                        props='color:white; background-color:red;',
                        subset=['overall'])

In [ ]:
FTS_NAME = 'fts_mra_tempmz'
# Get all the models with the above feature(s)
MODELS = [i for i in model_sum.index if (i.startswith(FTS_NAME) & 
                                         (len(i) < (len(FTS_NAME)+14)))]
# Get the selected models from model_sum
model_sum[model_sum.index.isin(MODELS)].sort_values(by='overall')\
             .style.highlight_min(axis=0, 
                                  props='color:darkblue; background-color:lightblue;',
                                  subset=target_labels_list)\
                    .highlight_min(axis=0, 
                                   props='color:white; background-color:red;',
                                   subset=['overall'])

In [ ]:
FTS_NAME = 'fts_mra_tempmz_cntpk_mratt'
# Get all the models with the above feature(s)
MODELS = [i for i in model_sum.index if (i.startswith(FTS_NAME) & 
                                         (len(i) < (len(FTS_NAME)+14)))]
# Get the selected models from model_sum
model_sum[model_sum.index.isin(MODELS)].sort_values(by='overall')\
             .style.highlight_min(axis=0, 
                                  props='color:darkblue; background-color:lightblue;',
                                  subset=target_labels_list)\
                    .highlight_min(axis=0, 
                                   props='color:white; background-color:red;',
                                   subset=['overall'])